# Question 3
#### Write a function that takes (ra,dec,distance,proper motion ra,proper motion dec,line-of-sight velocity) and their Gaussian uncertainties for a star and computes eccentricity, zmax, rperi, rap and their uncertainties using Monte Carlo sampling with the galpy function galpy.actionAngle.actionAngleStaeckel.EccZmaxRperiRap. Apply to some stars in the Gaia RV sample.

In [1]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
from galpy.potential import MWPotential2014
from galpy.actionAngle import actionAngleStaeckel

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


The following function takes the ICRS coordinates of a star, converts to galactocentric coordiantes, and calculates eccentricity, zmax, rperi, rap and their uncertainties. For the Monte Carlo sampling, I assumed that the ICRS coordinates are not correlated to each other (and hence the Gaussian distribution used for sampling is in the shape of a 5-sphere).

In [2]:
def icrs_to_EccZmaxRperiRap(ra: tuple, dec: tuple, d: tuple, pmra: tuple, 
                            pmdec: tuple, rv: tuple, ro: float = None, 
                            vo: float = None, nsamples: int = 100000) -> tuple:
    """ Converts ICRS coordinates and their Gaussian uncertainties for a star
    in the Milky War into eccentricity, maximum height above the plane, 
    pericentre, and apocentre with uncertainties.
    
    Parameters:
        
        ra - RA with uncertainty, given as a tuple of the form (RA, RA_ERR).
        Can be Quantity, otherwise given in degrees.
         
        dec - Dec with uncertainty, given as a tuple of the form 
        (DEC, DEC_ERR). Can be Quantity, otherwise given in degrees.
        
        d - Distance with uncertainty, given as a tuple of the form (D, D_ERR).
        Can be Quantity, otherwise given in kpc.
        
        pmra - Proper motion in RA with uncertainty, given as a tuple of the 
        form (PMRA, PMRA_ERR). Can be Quantity, otherwise given in mas/yr.
    
        pmdec - Proper motion in Dec with uncertainty, given as a tuple of the 
        form (PMDEC, PMDEC_ERR). Can be Quantity, otherwise given in mas/yr.
    
        rv - Radial velocity with uncertainty, given as a tuple of the form
        (RV, RV_ERR). Can be Quantity, otherwise given in km/s.
        
        ro (optional) - Distance scale. Can be Quantity, otherwise given in
        kpc. If provided with vo, activates output in physical units.
        
        vo (optional) - Velocity scale. Can be Quantity, otherwise given in 
        km/s. If provided with ro, activates output in physical units.
        
        nsamples (optional) - Number of samples for the Monte Carlo method.
    
    Returns:
        
        (e, zmax, rperi, rap), each a tuple of the form (VALUE, VALUE_ERR).
        If ro and vo are provided, distances are returned in kpc. Otherwise
        returns galpy natural units.
    """
    # Convert each value and its error into astropy Quantities
    ra_val, ra_err = u.Quantity(ra, u.deg)
    dec_val, dec_err = u.Quantity(dec, u.deg)
    d_val, d_err = u.Quantity(d, u.kpc)
    pmra_val, pmra_err = u.Quantity(pmra, u.mas/u.yr)
    pmdec_val, pmdec_err = u.Quantity(pmdec, u.mas/u.yr)
    rv_val, rv_err = u.Quantity(rv, u.km/u.s)
    
    # Get parameters for a 6D normal distribution to sample
    mean = np.array([ra_val.value, dec_val.value, d_val.value, pmra_val.value, 
                     pmdec_val.value, rv_val.value])
    cov = np.diag([ra_err.value, dec_err.value, d_err.value, pmra_err.value, 
                    pmdec_err.value, rv_err.value])
    
    # Sample the normal distribution
    samples = np.random.multivariate_normal(mean, cov, size=nsamples)
    ra_samples = samples[:, 0] * u.deg
    dec_samples = samples[:, 1] * u.deg
    d_samples = samples[:, 2] * u.kpc
    pmra_samples = samples[:, 3] * (u.mas/u.yr)
    pmdec_samples = samples[:, 4] * (u.mas/u.yr)
    rv_samples = samples[:, 5] * (u.km/u.s)
    
    coords = SkyCoord(frame="icrs", ra=ra_samples, dec=dec_samples,
                      distance=d_samples, pm_ra_cosdec=pmra_samples, 
                      pm_dec=pmdec_samples, radial_velocity=rv_samples)
    
    # Convert to galactocentric frame
    gal_coords = coords.transform_to("galactocentric")
    gal_coords.representation_type = "cylindrical"
    
    # Convert each value into consistent units
    R = gal_coords.rho.to(u.kpc)
    phi = gal_coords.phi.to(u.deg)
    z = gal_coords.z.to(u.kpc)
    vR = gal_coords.d_rho.to(u.km/u.s)
    vT = (gal_coords.d_phi * R).to(u.km/u.s, 
         equivalencies=u.dimensionless_angles())
    vz = gal_coords.d_z.to(u.km/u.s)
    
    # Calculate EccZmaxRperiRap
    aAS = actionAngleStaeckel(pot=MWPotential2014, delta=0.4, ro=ro, vo=vo)
    orbit_vals = aAS.EccZmaxRperiRap(R, vR, vT, z, vz, phi)
    
    # Get the mean and standard deviation of the samples
    ecc_val, zmax_val, rperi_val, rap_val = np.mean(orbit_vals, axis=1)
    ecc_err, zmax_err, rperi_err, rap_err = np.std(orbit_vals, axis=1)
    
    return ((ecc_val, ecc_err),
            (zmax_val, zmax_err),
            (rperi_val, rperi_err),
            (rap_val, rap_err))

We can grab a few stars from Gaia to test the function. I am approximating the uncertainty in distance using $\Delta d \approx \frac{\Delta p}{p^2}$. Since I am using this approximation, I am only querying for stars with small parallax uncertainties. Alternatively, the above function could be rewritten to accept parallax instead of distance for the Monte Carlo sampling.

In [3]:
import warnings
warnings.filterwarnings("ignore")

job = Gaia.launch_job_async("""
SELECT TOP 5
source_id, ra, ra_error, dec, dec_error, parallax, parallax_error, pmra, 
pmra_error, pmdec, pmdec_error, radial_velocity, radial_velocity_error
FROM gaiadr2.gaia_source 
WHERE radial_velocity IS NOT NULL
AND ABS(radial_velocity) < 100.0
AND parallax_error < parallax * 0.1
""")
stars = job.get_results()

for i in range(len(stars)):
    source_id = stars[i]["source_id"]
    ra = (stars[i]["ra"], stars[i]["ra_error"])
    dec = (stars[i]["dec"], stars[i]["dec_error"])
    pmra = (stars[i]["pmra"], stars[i]["pmra_error"])
    pmdec = (stars[i]["pmdec"], stars[i]["pmdec_error"])
    rv = (stars[i]["radial_velocity"], stars[i]["radial_velocity_error"])
    
    # Calculate distance and its approximate uncertainty from parallax
    d_val = 1/stars[i]["parallax"]
    d_err = abs(stars[i]["parallax_error"]/(stars[i]["parallax"]**2))
    d = (d_val, d_err)
    
    ecc, zmax, rperi, rap = icrs_to_EccZmaxRperiRap(ra, dec, d, pmra, pmdec, 
                                                    rv, ro=8., vo=220.)
    print("\nsource_id: {}".format(source_id))
    print("e = " + "{} +/- {}".format(*ecc))
    print("zmax = " + "{} +/- {} kpc".format(*zmax))
    print("rperi = " + "{} +/- {} kpc".format(*rperi))
    print("rap = " + "{} +/- {} kpc".format(*rap))

Query finished.

source_id: 6216803532750768128
e = 0.2824356422122744 +/- 0.007569353518518249
zmax = 1.3648740681860565 +/- 0.1223256756717155 kpc
rperi = 5.770446578962753 +/- 0.4352745937277204 kpc
rap = 10.312320813474797 +/- 0.7777192902580037 kpc

source_id: 430087721411804416
e = 0.32415428871354 +/- 0.011654231954122537
zmax = 0.25836506091909284 +/- 0.08049405146849194 kpc
rperi = 5.137866925816945 +/- 0.22590907478149158 kpc
rap = 10.060417940930904 +/- 0.2299609268971258 kpc

source_id: 1730739878892706176
e = 0.43631732601752077 +/- 0.044634631001340005
zmax = 1.1224280807437068 +/- 0.16396179319658177 kpc
rperi = 4.00712131260837 +/- 0.22907673170480627 kpc
rap = 10.236996995539597 +/- 0.5691309236507143 kpc

source_id: 4099060334262501760
e = 0.298986504295441 +/- 0.004806961236947552
zmax = 0.38278773233144986 +/- 0.014487553266968715 kpc
rperi = 5.722171505357887 +/- 0.04683624212457421 kpc
rap = 10.603185184316567 +/- 0.04844740775587819 kpc

source_id: 60190869810247